# Identifying potential Bot or Paid reviewers using Amazon customer review data using Network Analysis
**By: SLT11A**
* Abel Johua Cruzada
* James Gonzales

# Executive Summary

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx
import networkx_abel as nxa

# Introduction

# Problem Statement

# Objectives

# Data Description

# Methodology

# Data Preprocessing

Only include data from 2015 onwards.

In [11]:
df = pd.read_csv('amazon_reviews_video_games.csv', index_col=0)

In [17]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12039526,RTIS3L2M1F5SM,B001CXYMFS,737716809,Thrustmaster T-Flight Hotas X Flight Stick,Video Games,5,0,0,N,Y,an amazing joystick. I especially love that yo...,"Used this for Elite Dangerous on my mac, an am...",2015-08-31
1,US,9636577,R1ZV7R40OLHKD,B00M920ND6,569686175,Tonsee 6 buttons Wireless Optical Silent Gamin...,Video Games,5,0,0,N,Y,Definitely a silent mouse... Not a single clic...,"Loved it, I didn't even realise it was a gami...",2015-08-31
2,US,2331478,R3BH071QLH8QMC,B0029CSOD2,98937668,Hidden Mysteries: Titanic Secrets of the Fatef...,Video Games,1,0,1,N,Y,One Star,poor quality work and not as it is advertised.,2015-08-31
3,US,52495923,R127K9NTSXA2YH,B00GOOSV98,23143350,GelTabz Performance Thumb Grips - PlayStation ...,Video Games,3,0,0,N,Y,"good, but could be bettee","nice, but tend to slip away from stick in inte...",2015-08-31
4,US,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,Video Games,4,0,0,N,Y,Great but flawed.,"Great amiibo, great for collecting. Quality ma...",2015-08-31


Remove single value columns

In [21]:
df.nunique()

marketplace               1
customer_id          219045
review_id            330966
product_id            32693
product_parent        31366
product_title         27053
product_category          1
star_rating               5
helpful_votes           183
total_votes             246
vine                      2
verified_purchase         2
review_headline      137990
review_body          259517
review_date             243
dtype: int64

In [22]:
df.drop(['marketplace', 'product_category'], axis=1, inplace=True)

## Text Preprocessing

* Remove Stopwords, High Frequency Words.
* Tokenization

# EDA

# FIM

customer1: {product1, product2, product3},

customer2: {product1, product2, product4}

## Confidence, Lift, Covariance

# Network

## Construct the Network

Color code based on 5 rating, scale the size of the node based on number of degrees, change the weight of the links based on prevalence of reviewed products.

## Network Properties

## Centrality Measurements

## Community Detection

# Review Information Network

Co-occurrence of words in the review.

## Cosine Similarity

## Clustering

## Customer Review Frame Network

# Conclusion

# Recommendation

# Reference